#     Baseline Training and Testing using the unpreprocessed dataset

In [1]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from statistics import mean

from sklearn import linear_model, tree, svm
from sklearn.datasets import load_digits
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve
from sklearn.datasets import load_wine
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
df = pd.read_csv('../data/JMpreprocessed_data2.csv')

In [3]:
df.head()

,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,1,0.046875,0.070312,0.552969,0.412422,0.207031,0.152188,0.000000,0.000000,0.000000,...,0,1,0,0,0,0,1,0,0,0
1,0,0.001991,0.289082,0.444719,0.516684,0.141343,0.000017,0.016671,0.023640,0.007814,...,0,1,0,0,0,0,0,0,0,1
2,1,0.039551,0.008089,0.375617,0.431954,0.196597,0.330535,0.002093,1.318322,0.000015,...,0,1,0,0,0,0,0,0,1,0
3,0,0.015625,0.000000,0.450000,0.428447,0.250000,0.200000,0.148449,0.125000,0.000000,...,0,0,0,1,0,0,1,0,0,0
4,1,0.759773,0.019547,0.588316,0.504957,0.248394,0.002326,0.750267,0.126350,0.000000,...,0,0,0,1,0,0,1,0,0,0


In [4]:
#splitting up data into training/testing
y = df['Winner']
X = df.drop(columns = 'Winner')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=43)

In [5]:
#Creating Model Instances
logistic_regression_model = LogisticRegression(max_iter=50000)
adaboost_model = AdaBoostClassifier(n_estimators=500, random_state=0)#changed from 100 to 500
ann_model = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10, 10, 10, 5, 5, 5, 2), random_state=1, max_iter=350, learning_rate='adaptive')
dtree_model = tree.DecisionTreeClassifier()
#naivebayesgaussian_model = GaussianNB()
randforest_model = RandomForestClassifier(max_depth=5, random_state=0)
svm_model = svm.SVC(decision_function_shape='ovo')
gradient_boosting_model = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=5)#3 -7. 100- 200
xgboost_model = xgb.XGBClassifier(max_depth=12,
                        subsample=0.33,
                        objective='binary:logistic',
                        n_estimators=300,
                        learning_rate = 0.01)

# Training/Testing

In [6]:
'''
This function trains/test and returns the accuracy of that iteration
'''
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    
    y_pred=model.predict(x_test)
    print(confusion_matrix(y_test, y_pred))
    return model.score(x_test, y_test)

In [7]:
get_score(xgboost_model, X_train, X_test, y_train, y_test)

/Users/justinmilner/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:53:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[183  16]
 [ 76  17]]


0.684931506849315

In [8]:
get_score(logistic_regression_model, X_train, X_test, y_train, y_test)

[[173  26]
 [ 67  26]]


/Users/justinmilner/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6815068493150684

In [9]:
get_score(svm_model, X_train, X_test, y_train, y_test)

[[199   0]
 [ 93   0]]


0.6815068493150684

In [10]:
get_score(ann_model, X_train, X_test, y_train, y_test)

[[182  17]
 [ 78  15]]


0.6746575342465754

In [11]:
get_score(adaboost_model, X_train, X_test, y_train, y_test)

[[164  35]
 [ 51  42]]


0.7054794520547946

In [12]:
get_score(dtree_model, X_train, X_test, y_train, y_test)

[[136  63]
 [ 55  38]]


0.5958904109589042

In [13]:
get_score(randforest_model, X_train, X_test, y_train, y_test)

[[199   0]
 [ 92   1]]


0.684931506849315

In [14]:
get_score(gradient_boosting_model, X_train, X_test, y_train, y_test)

[[175  24]
 [ 70  23]]


0.678082191780822